# US Equity Indices Data Collection

This notebook retrieves comprehensive US equity market data including:
- Major market indices (S&P 500, Nasdaq Composite, Russell 2000)
- Sector indices for each major index
- Growth vs Value indices for each major index

In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
import certifi
import json
import os
import ssl
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Environment variables
import dotenv
dotenv.load_dotenv()
FMP_API_KEY = os.getenv('FMP_API_KEY')

## Define US Equity Index Symbols

Comprehensive list of major indices, their sectors, and growth/value variants

In [2]:
# US Equity Indices Dictionary
us_equity_indices = {
    'US Government Bonds': {
        'main': 'BIL', 
        'name': 'SPDR Bloomberg 1-3 Month T-Bill ETF',
        'etfs': {
                'SHY': 'iShares 1-3 Year Treasury Bond ETF',
                'IEF': 'iShares 7-10 Year Treasury Bond ETF',
            }
    },
    'International Equitiy ETFs': {
        'main': 'VXUS', 
        'name': 'Vanguard Total International Stock ETF',
        'etfs': {
                'VEA': 'Vanguard FTSE Developed Markets ETF',
                'VWO': 'Vanguard FTSE Emerging Markets ETF',
                'VGK': 'Vanguard FTSE Europe ETF',
                'VPL': 'Vanguard FTSE Pacific ETF',
                'FXI': 'iShares China Large-Cap ETF',
                'EWJ': 'iShares MSCI Japan ETF',
                'INDA': 'iShares MSCI India ETF',
            }
    },
    # ============================================================================
    # S&P 500 FAMILY
    # ============================================================================
    'SP500': {
        'main': '^GSPC',
        'name': 'S&P 500 Index',
        'etfs': {
            'SPY': 'SPDR S&P 500 ETF Trust',
            'VOO': 'Vanguard S&P 500 ETF',
            'RSP': 'Invesco S&P 500 Equal Weight ETF'
        },
        'sectors': {
            'XLE': 'Energy Select Sector SPDR',
            'XLF': 'Financial Select Sector SPDR',
            'XLU': 'Utilities Select Sector SPDR',
            'XLI': 'Industrial Select Sector SPDR',
            'XLV': 'Health Care Select Sector SPDR',
            'XLK': 'Technology Select Sector SPDR',
            'XLB': 'Materials Select Sector SPDR',
            'XLY': 'Consumer Discretionary Select Sector SPDR',
            'XLP': 'Consumer Staples Select Sector SPDR',
            'XLRE': 'Real Estate Select Sector SPDR',
            'XLC': 'Communication Services Select Sector SPDR'
        },
        'style': {
            'IVW': 'iShares S&P 500 Growth ETF',
            'IVE': 'iShares S&P 500 Value ETF',
        }
    },
    
    # ============================================================================
    # NASDAQ COMPOSITE & NASDAQ 100
    # ============================================================================
    'NASDAQ': {
        'main': '^IXIC',  # Nasdaq Composite
        'name': 'Nasdaq Composite Index',
        'etfs': {
            'ONEQ': 'Fidelity Nasdaq Composite Index ETF'
        }
    },

    'NASDAQ 100': {
        'main': '^NDX',
        'name': 'Nasdaq 100 Index',
        'etfs': {
            'QQQ': 'Invesco QQQ Trust',
        },
    },
    
    # ============================================================================
    # RUSSELL 2000 (SMALL CAP)
    # ============================================================================
    'RUSSELL2000': {
        'main': '^RUT',
        'name': 'Russell 2000 Index',
        'etfs': {
            'IWM': 'iShares Russell 2000 ETF',
        },
        'size': {
            'IWR': 'iShares Russell Mid-Cap ETF',
            'IWC': 'iShares Micro-Cap ETF',
        },
        'style': {
            'IWO': 'iShares Russell 2000 Growth ETF',
            'IWN': 'iShares Russell 2000 Value ETF',

        }
    },
    
    # ============================================================================
    # ADDITIONAL MAJOR US INDICES
    # ============================================================================
    'DOW': {
        'main': '^DJI',
        'name': 'Dow Jones Industrial Average',
        'etfs': {
            'DIA': 'SPDR Dow Jones Industrial Average ETF Trust'
        }
    },
    
    'RUSSELL1000': {
        'main': '^RUI',
        'name': 'Russell 1000 Index',
        'etfs': {
            'IWB': 'iShares Russell 1000 ETF',
        },
        'style': {
            'IWF': 'iShares Russell 1000 Growth ETF',
            'IWD': 'iShares Russell 1000 Value ETF',
        }
    },
    
    'RUSSELL3000': {
        'main': '^RUA',
        'name': 'Russell 3000 Index',
        'etfs': {
            'IWV': 'iShares Russell 3000 ETF',
        },
        'style': {
            'VTHR': 'Vanguard Russell 3000 ETF',
            'THRK': 'iShares Russell 3000 ETF'
        }
    },
}

print('US Equity Indices Dictionary Created')
print(f'Total Major Indices: {len(us_equity_indices)}')
total_symbols = sum([
    1 + len(idx.get('sectors', {})) + len(idx.get('style', {})) 
    for idx in us_equity_indices.values()
])
print(f'Total Symbols to Retrieve: {total_symbols}')

US Equity Indices Dictionary Created
Total Major Indices: 9
Total Symbols to Retrieve: 28


## Helper Functions for Data Retrieval

In [3]:
def get_jsonparsed_data(url):
    """Fetch JSON data from URL with SSL certificate verification"""
    context = ssl.create_default_context(cafile=certifi.where())
    response = urlopen(url, context=context)
    data = response.read().decode('utf-8')
    return json.loads(data)


def fetch_index_data(symbol, date_from='1990-01-01', date_to='2025-09-26', api_key=FMP_API_KEY):
    """
    Fetch historical OHLCV data for a given symbol from FMP API
    
    Parameters:
    -----------
    symbol : str
        Ticker symbol (e.g., '^GSPC', 'SPY')
    date_from : str
        Start date in YYYY-MM-DD format
    date_to : str
        End date in YYYY-MM-DD format
    api_key : str
        FMP API key
    
    Returns:
    --------
    pd.DataFrame with MultiIndex columns [symbol, field]
    """
    try:
        # NOTE: FMP API may require different endpoint for indices vs ETFs
        # Indices typically use ^SYMBOL notation
        url = f'https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?from={date_from}&to={date_to}&apikey={api_key}'
        
        data = get_jsonparsed_data(url)
        
        if 'historical' not in data:
            print(f'Warning: No historical data found for {symbol}')
            return None
        
        df = pd.DataFrame(data['historical'])
        
        # Select and order columns
        df = df[['date', 'open', 'high', 'low', 'close', 'volume']].copy()
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date').sort_index()
        
        # Add vwap if not present (approximate as average of high and low)
        if 'vwap' not in df.columns:
            df['vwap'] = (df['high'] + df['low']) / 2
        
        # Create MultiIndex columns with symbol name
        df.columns = pd.MultiIndex.from_product([[symbol], df.columns])
        
        print(f'✓ {symbol}: {len(df)} observations from {df.index.min().date()} to {df.index.max().date()}')
        return df
        
    except Exception as e:
        print(f'✗ {symbol}: Error - {str(e)}')
        return None

def fetch_all_us_equity_data(indices_dict, date_from='1990-01-01', date_to='2025-09-26'):
    """
    Fetch all US equity indices data from the indices dictionary with improved categorization

    Parameters:
    -----------
    indices_dict : dict
        Dictionary of US equity indices with structure:
        {
            'INDEX_NAME': {
                'main': 'ticker',        # Main index (e.g., '^GSPC')
                'name': 'Full Name',
                'etfs': {...},           # Main ETFs tracking the index
                'sectors': {...},        # Sector-specific ETFs/indices
                'style': {...},          # Growth/Value style ETFs
                'size': {...}            # Size-specific ETFs (optional)
            }
        }
    date_from : str
        Start date in YYYY-MM-DD format
    date_to : str
        End date in YYYY-MM-DD format

    Returns:
    --------
    tuple: (main_indices_df, main_etfs_df, sector_etfs_df, style_etfs_df, size_etfs_df, metadata_df)
        - main_indices_df: Actual market indices (e.g., ^GSPC, ^IXIC)
        - main_etfs_df: Primary ETFs tracking main indices (e.g., SPY, QQQ)
        - sector_etfs_df: Sector-specific ETFs (e.g., XLE, XLF)
        - style_etfs_df: Growth/Value style ETFs (e.g., IVW, IVE)
        - size_etfs_df: Market cap size ETFs (e.g., IWM, IWR)
        - metadata_df: Complete metadata for all symbols
    """
    # Separate storage for different categories
    main_indices_list = []
    main_etfs_list = []
    sector_etfs_list = []
    style_etfs_list = []
    size_etfs_list = []
    metadata = []

    for index_family, index_info in indices_dict.items():
        print(f'\n{"="*60}')
        print(f'Processing {index_family}: {index_info["name"]}')
        print(f'{"="*60}')

        # ========================================================================
        # 1. Fetch Main Index (actual market index, not ETF)
        # ========================================================================
        print(f'\n[1/5] Main Index')
        main_symbol = index_info['main']
        main_data = fetch_index_data(main_symbol, date_from, date_to)
        if main_data is not None:
            main_indices_list.append(main_data)
            metadata.append({
                'symbol': main_symbol,
                'name': index_info['name'],
                'category': 'main_index',
                'subcategory': 'index',
                'family': index_family
            })

        # ========================================================================
        # 2. Fetch Main ETFs (primary ETFs that track the main index)
        # ========================================================================
        if 'etfs' in index_info and index_info['etfs']:
            print(f'\n[2/5] Main ETFs - {len(index_info["etfs"])} ETFs')
            for symbol, name in index_info['etfs'].items():
                etf_data = fetch_index_data(symbol, date_from, date_to)
                if etf_data is not None:
                    main_etfs_list.append(etf_data)
                    metadata.append({
                        'symbol': symbol,
                        'name': name,
                        'category': 'main_etf',
                        'subcategory': 'broad_market',
                        'family': index_family
                    })
        else:
            print(f'\n[2/5] Main ETFs - None defined')

        # ========================================================================
        # 3. Fetch Sector ETFs
        # ========================================================================
        if 'sectors' in index_info and index_info['sectors']:
            print(f'\n[3/5] Sector ETFs - {len(index_info["sectors"])} sectors')
            for symbol, name in index_info['sectors'].items():
                sector_data = fetch_index_data(symbol, date_from, date_to)
                if sector_data is not None:
                    sector_etfs_list.append(sector_data)
                    # Extract sector name from ETF name if possible
                    sector_name = name.split('Select Sector')[0].strip() if 'Select Sector' in name else name
                    metadata.append({
                        'symbol': symbol,
                        'name': name,
                        'category': 'sector_etf',
                        'subcategory': sector_name,
                        'family': index_family
                    })
        else:
            print(f'\n[3/5] Sector ETFs - None defined')

        # ========================================================================
        # 4. Fetch Style ETFs (Growth/Value)
        # ========================================================================
        if 'style' in index_info and index_info['style']:
            print(f'\n[4/5] Style ETFs - {len(index_info["style"])} styles')
            for symbol, name in index_info['style'].items():
                style_data = fetch_index_data(symbol, date_from, date_to)
                if style_data is not None:
                    style_etfs_list.append(style_data)
                    # Determine if Growth or Value
                    style_type = 'Growth' if 'Growth' in name else 'Value' if 'Value' in name else 'Blend'
                    metadata.append({
                        'symbol': symbol,
                        'name': name,
                        'category': 'style_etf',
                        'subcategory': style_type,
                        'family': index_family
                    })
        else:
            print(f'\n[4/5] Style ETFs - None defined')

        # ========================================================================
        # 5. Fetch Size ETFs (Small/Mid/Large Cap)
        # ========================================================================
        if 'size' in index_info and index_info['size']:
            print(f'\n[5/5] Size ETFs - {len(index_info["size"])} size categories')
            for symbol, name in index_info['size'].items():
                size_data = fetch_index_data(symbol, date_from, date_to)
                if size_data is not None:
                    size_etfs_list.append(size_data)
                    # Determine size category
                    if 'Mid' in name or 'Mid-Cap' in name:
                        size_cat = 'Mid-Cap'
                    elif 'Small' in name or 'Small-Cap' in name:
                        size_cat = 'Small-Cap'
                    elif 'Large' in name or 'Large-Cap' in name:
                        size_cat = 'Large-Cap'
                    elif 'Micro' in name:
                        size_cat = 'Micro-Cap'
                    else:
                        size_cat = 'Other'

                    metadata.append({
                        'symbol': symbol,
                        'name': name,
                        'category': 'size_etf',
                        'subcategory': size_cat,
                        'family': index_family
                    })
        else:
            print(f'\n[5/5] Size ETFs - None defined')

    # ========================================================================
    # Concatenate all dataframes by category
    # ========================================================================
    print(f'\n{"="*60}')
    print('COMBINING ALL DATA BY CATEGORY')
    print(f'{"="*60}')

    main_indices_df = pd.concat(main_indices_list, axis=1) if main_indices_list else None
    main_etfs_df = pd.concat(main_etfs_list, axis=1) if main_etfs_list else None
    sector_etfs_df = pd.concat(sector_etfs_list, axis=1) if sector_etfs_list else None
    style_etfs_df = pd.concat(style_etfs_list, axis=1) if style_etfs_list else None
    size_etfs_df = pd.concat(size_etfs_list, axis=1) if size_etfs_list else None
    metadata_df = pd.DataFrame(metadata)

    # Print summary statistics
    if main_indices_df is not None:
        print(f'\nMain Indices: {main_indices_df.shape[0]} rows × {len(main_indices_list)} indices')
    if main_etfs_df is not None:
        print(f'Main ETFs: {main_etfs_df.shape[0]} rows × {len(main_etfs_list)} ETFs')
    if sector_etfs_df is not None:
        print(f'Sector ETFs: {sector_etfs_df.shape[0]} rows × {len(sector_etfs_list)} ETFs')
    if style_etfs_df is not None:
        print(f'Style ETFs: {style_etfs_df.shape[0]} rows × {len(style_etfs_list)} ETFs')
    if size_etfs_df is not None:
        print(f'Size ETFs: {size_etfs_df.shape[0]} rows × {len(size_etfs_list)} ETFs')

    print(f'\nTotal symbols retrieved: {len(metadata)}')
    print(f'Date range: {date_from} to {date_to}')

    return main_indices_df, main_etfs_df, sector_etfs_df, style_etfs_df, size_etfs_df, metadata_df

us_equity_symbol_names = {
    # BOND ETFS
    'BIL': 'SPDR Bloomberg 1-3 Month T-Bill ETF',
    'SHY': 'iShares 1-3 Year Treasury Bond ETF',
    'IEF': 'iShares 7-10 Year Treasury Bond ETF',
    # MAJOR INDICES
    '^GSPC': 'S&P 500',
    '^IXIC': 'Nasdaq Composite',
    '^NDX': 'Nasdaq 100',
    '^RUT': 'Russell 2000',
    '^DJI': 'Dow Jones Industrial Average',
    '^RUI': 'Russell 1000',
    '^RUA': 'Russell 3000',
    
    # MAIN BROAD MARKET ETFS
    'SPY': 'SPDR S&P 500 ETF',
    'VOO': 'Vanguard S&P 500 ETF',
    'RSP': 'Invesco S&P 500 Equal Weight ETF',
    'IVV': 'iShares Core S&P 500 ETF',
    'QQQ': 'Invesco QQQ Trust',
    'QQQM': 'Invesco Nasdaq 100 ETF',
    'ONEQ': 'Fidelity Nasdaq Composite Index ETF',
    'IWM': 'iShares Russell 2000 ETF',
    'IWB': 'iShares Russell 1000 ETF',
    'IWV': 'iShares Russell 3000 ETF',
    'DIA': 'SPDR Dow Jones Industrial Average ETF',
    'VTI': 'Vanguard Total Stock Market ETF',
    
    # S&P 500 SECTOR ETFS (SELECT SECTOR SPDRS)
    'XLE': 'Energy Select Sector SPDR',
    'XLF': 'Financial Select Sector SPDR',
    'XLU': 'Utilities Select Sector SPDR',
    'XLI': 'Industrial Select Sector SPDR',
    'XLV': 'Health Care Select Sector SPDR',
    'XLK': 'Technology Select Sector SPDR',
    'XLB': 'Materials Select Sector SPDR',
    'XLY': 'Consumer Discretionary Select Sector SPDR',
    'XLP': 'Consumer Staples Select Sector SPDR',
    'XLRE': 'Real Estate Select Sector SPDR',
    'XLC': 'Communication Services Select Sector SPDR',
    
    # GROWTH ETFs
    'IVW': 'iShares S&P 500 Growth ETF',
    'VONG': 'Vanguard Russell 1000 Growth ETF',
    'IWF': 'iShares Russell 1000 Growth ETF',
    'IWO': 'iShares Russell 2000 Growth ETF',
    'VUG': 'Vanguard Growth ETF',
    'SPYG': 'SPDR Portfolio S&P 500 Growth ETF',
    
    # VALUE ETFs
    'IVE': 'iShares S&P 500 Value ETF',
    'VONV': 'Vanguard Russell 1000 Value ETF',
    'IWD': 'iShares Russell 1000 Value ETF',
    'IWN': 'iShares Russell 2000 Value ETF',
    'VTV': 'Vanguard Value ETF',
    'SPYV': 'SPDR Portfolio S&P 500 Value ETF',
    
    # SIZE ETFs
    'IWR': 'iShares Russell Mid-Cap ETF',
    'IWC': 'iShares Micro-Cap ETF',
    'IJH': 'iShares Core S&P Mid-Cap ETF',
    'IJR': 'iShares Core S&P Small-Cap ETF',
    'MDY': 'SPDR S&P MidCap 400 ETF',
    'SLY': 'SPDR S&P 600 Small Cap ETF',
    'VO': 'Vanguard Mid-Cap ETF',
    'VB': 'Vanguard Small-Cap ETF',
    'SCHA': 'Schwab U.S. Small-Cap ETF',
    'SCHM': 'Schwab U.S. Mid-Cap ETF',
    'VTWO': 'Vanguard Russell 2000 ETF',
    'VTHR': 'Vanguard Russell 3000 ETF',
    'THRK': 'iShares Russell 3000 ETF',
    'SPSM': 'SPDR Portfolio S&P 600 Small Cap ETF',
    'SMLF': 'iShares Small-Cap US Equity Factor ETF',
    
    # NASDAQ SPECIFIC
    'QTEC': 'First Trust Nasdaq-100 Technology Sector Index Fund',
    'QQEW': 'First Trust Nasdaq-100 Equal Weighted Index Fund',
    'QQQG': 'Pacer Nasdaq 100 Top 50 Cash Cows Dividend Growth ETF',
    'QQQV': 'Pacer Nasdaq 100 Top 50 Value ETF',
    
    # DIVIDEND/QUALITY
    'SCHD': 'Schwab U.S. Dividend Equity ETF',
    'VYM': 'Vanguard High Dividend Yield ETF',
    'DVY': 'iShares Select Dividend ETF',
    'QUAL': 'iShares MSCI USA Quality Factor ETF',
    'USMV': 'iShares MSCI USA Min Vol Factor ETF',
    
    # EQUAL WEIGHT
    'EWSC': 'Invesco S&P SmallCap 600 Equal Weight ETF',
    'EWMC': 'Invesco S&P MidCap 400 Equal Weight ETF',
}


## Fetch US Equity Data

In [4]:
# Set date range
date_from = '1990-01-01'
date_to = '2025-09-26'

# Fetch all data with new categorization
main_indices_df, main_etfs_df, sector_etfs_df, style_etfs_df, size_etfs_df, metadata_df = fetch_all_us_equity_data(
    us_equity_indices, 
    date_from=date_from, 
    date_to=date_to
)

print('\n' + '='*60)
print('DATA RETRIEVAL COMPLETE')
print('='*60)
print('\nData Structure:')
print(f'  1. Main Indices (^GSPC, ^IXIC, etc.): {main_indices_df.shape if main_indices_df is not None else "None"}')
print(f'  2. Main ETFs (SPY, QQQ, etc.): {main_etfs_df.shape if main_etfs_df is not None else "None"}')
print(f'  3. Sector ETFs (XLE, XLF, etc.): {sector_etfs_df.shape if sector_etfs_df is not None else "None"}')
print(f'  4. Style ETFs (IVW, IVE, etc.): {style_etfs_df.shape if style_etfs_df is not None else "None"}')
print(f'  5. Size ETFs (IWM, IWR, etc.): {size_etfs_df.shape if size_etfs_df is not None else "None"}')
print(f'  6. Metadata: {metadata_df.shape}')


Processing US Government Bonds: SPDR Bloomberg 1-3 Month T-Bill ETF

[1/5] Main Index
✓ BIL: 4613 observations from 2007-05-30 to 2025-09-26

[2/5] Main ETFs - 2 ETFs
✓ SHY: 5831 observations from 2002-07-26 to 2025-09-26
✓ IEF: 5831 observations from 2002-07-26 to 2025-09-26

[3/5] Sector ETFs - None defined

[4/5] Style ETFs - None defined

[5/5] Size ETFs - None defined

Processing International Equitiy ETFs: Vanguard Total International Stock ETF

[1/5] Main Index
✓ VXUS: 3688 observations from 2011-01-28 to 2025-09-26

[2/5] Main ETFs - 7 ETFs
✓ VEA: 4573 observations from 2007-07-26 to 2025-09-26
✓ VWO: 5171 observations from 2005-03-10 to 2025-09-26
✓ VGK: 5171 observations from 2005-03-10 to 2025-09-26
✓ VPL: 5171 observations from 2005-03-10 to 2025-09-26
✓ FXI: 5276 observations from 2004-10-08 to 2025-09-26
✓ EWJ: 7431 observations from 1996-03-18 to 2025-09-26
✓ INDA: 3432 observations from 2012-02-03 to 2025-09-26

[3/5] Sector ETFs - None defined

[4/5] Style ETFs - None

## Save Retrieved Data

In [5]:
# Save all data categories separately
if main_indices_df is not None:
    main_indices_df.to_csv('data/processed/us_equity_main_indices.csv')
    print('✓ Main indices saved to: data/processed/us_equity_main_indices.csv')

if main_etfs_df is not None:
    main_etfs_df.to_csv('data/processed/us_equity_main_etfs.csv')
    print('✓ Main ETFs saved to: data/processed/us_equity_main_etfs.csv')

if sector_etfs_df is not None:
    sector_etfs_df.to_csv('data/processed/us_equity_sector_etfs.csv')
    print('✓ Sector ETFs saved to: data/processed/us_equity_sector_etfs.csv')

if style_etfs_df is not None:
    style_etfs_df.to_csv('data/processed/us_equity_style_etfs.csv')
    print('✓ Style ETFs saved to: data/processed/us_equity_style_etfs.csv')

if size_etfs_df is not None:
    size_etfs_df.to_csv('data/processed/us_equity_size_etfs.csv')
    print('✓ Size ETFs saved to: data/processed/us_equity_size_etfs.csv')

# # Save metadata
# metadata_df.to_csv('data/processed/us_equity_metadata.csv', index=False)
# print('✓ Metadata saved to: data/processed/us_equity_metadata.csv')

# Combine all into single comprehensive file
all_us_equity_data = pd.concat([
    df for df in [main_indices_df, main_etfs_df, sector_etfs_df, style_etfs_df, size_etfs_df] 
    if df is not None
], axis=1)

all_us_equity_data.to_csv('data/processed/us_equity_all_data.csv')
print('✓ All US equity data saved to: data/processed/us_equity_all_data.csv')
print(f'\nFinal combined dataset: {all_us_equity_data.shape[0]} rows × {all_us_equity_data.shape[1]} columns')
print(f'Date range: {all_us_equity_data.index.min().date()} to {all_us_equity_data.index.max().date()}')

✓ Main indices saved to: data/processed/us_equity_main_indices.csv
✓ Main ETFs saved to: data/processed/us_equity_main_etfs.csv
✓ Sector ETFs saved to: data/processed/us_equity_sector_etfs.csv
✓ Style ETFs saved to: data/processed/us_equity_style_etfs.csv
✓ Size ETFs saved to: data/processed/us_equity_size_etfs.csv
✓ All US equity data saved to: data/processed/us_equity_all_data.csv

Final combined dataset: 9002 rows × 282 columns
Date range: 1990-01-02 to 2025-09-26


## Load Saved US Equity Data

In [6]:
all_us_equity_data = pd.read_csv('data/processed/us_equity_all_data.csv', index_col=0, header=[0, 1], parse_dates=True)
all_us_equity_close_cols = all_us_equity_data.columns[all_us_equity_data.columns.get_level_values(1) == 'close']
all_us_equity_close_prices = all_us_equity_data[all_us_equity_close_cols].droplevel(1, axis=1).rename(columns=us_equity_symbol_names)
all_us_equity_close_prices

,SPDR Bloomberg 1-3 Month T-Bill ETF,VXUS,S&P 500,Nasdaq Composite,Nasdaq 100,Russell 2000,Dow Jones Industrial Average,Russell 1000,Russell 3000,iShares 1-3 Year Treasury Bond ETF,...,Communication Services Select Sector SPDR,iShares S&P 500 Growth ETF,iShares S&P 500 Value ETF,iShares Russell 2000 Growth ETF,iShares Russell 2000 Value ETF,iShares Russell 1000 Growth ETF,iShares Russell 1000 Value ETF,Vanguard Russell 3000 ETF,iShares Russell Mid-Cap ETF,iShares Micro-Cap ETF
date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,359.69,459.30,227.73,169.949997,2810.149902,188.03,200.309998,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,358.76,460.90,226.10,170.789993,2809.729980,187.75,200.110001,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,355.67,459.40,224.71,170.089996,2796.080078,186.25,198.850006,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,352.20,458.20,223.14,169.649994,2773.250000,184.58,196.860001,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,353.79,458.70,223.54,169.550003,2794.370117,185.23,197.539993,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-22,91.69,73.44,6693.74,22788.98,24761.07,2463.340000,46381.550000,3663.52,3812.540000,82.92,...,118.58,121.65,204.98,324.95,177.46,472.68,202.48,295.85,96.45,151.39
2025-09-23,91.69,73.34,6656.93,22573.47,24580.17,2457.510000,46292.790000,3643.31,3792.050000,82.95,...,118.41,120.41,205.06,324.07,177.35,467.74,202.63,294.16,96.46,150.85
2025-09-24,91.71,72.91,6637.98,22497.86,24503.57,2434.980000,46121.290000,3631.20,3778.450000,82.91,...,117.43,119.90,204.68,319.38,176.86,465.22,202.23,292.30,95.89,149.83
